# SCRAP B3

Pipeline Batch Bovespa (entrega obrigatória):

- ✅ Requisito 1: scrap de dados do site da B3 com dados do pregão.
- ⬜ Requisito 2: os dados brutos devem ser ingeridos no s3 em formato parquet com partição diária.
- ⬜ Requisito 3: o bucket deve acionar uma lambda, que por sua vez irá chamar o job de ETL no glue.
- ⬜ Requisito 4: a lambda pode ser em qualquer linguagem. Ela apenas deverá iniciar o job Glue.
- ⬜ Requisito 5: o job Glue deve ser feito no modo visual. Este job deve conter as seguintes transformações obrigatórias:
  - A: agrupamento numérico, sumarização, contagem ou soma.
  - B: renomear duas colunas existentes além das de agrupamento.
  - C: realizar um cálculo com campos de data, exemplo, poder ser duração, comparação, diferença entre datas.
- ⬜ Requisito 6: os dados refinados no job glue devem ser salvos no formato parquet em uma pasta chamada refined, particionado por data e pelo nome ou abreviação da ação do pregão.
- ⬜ Requisito 7: o job Glue deve automaticamente catalogar o dado no Glue Catalog e criar uma tabela no banco de dados default do Glue Catalog.
- ⬜ Requisito 8: os dados devem estar disponíveis e legíveis no Athena.
- ⬜ Requisito 9: é opcional construir um notebook no Athena para montar uma visualização gráfica dos dados ingeridos.


### LINK OBRIGATÓRIO:
#### https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br

# Requisito 1: scrap de dados do site da B3 com dados do pregão.

In [1]:
import requests
import json

# Payload atualizado para consulta por setor com pageSize=20 e segment=2 (exemplo)
payload = "eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjEyMCwiaW5kZXgiOiJJQk9WIiwic2VnbWVudCI6IjIifQ=="
url = f"https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/{payload}"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json"
}

resp = requests.get(url, headers=headers)
resp.raise_for_status()
data = resp.json()

registros = data.get("results") or data
if isinstance(registros, dict):
    # tentar pegar lista dentro do dict
    registros = next((v for v in registros.values() if isinstance(v, list)), [data])

print(f"Recebidos {len(registros)} registros.")
print(json.dumps(registros[:2], ensure_ascii=False, indent=2))  # imprime só os 2 primeiros para exemplo


Recebidos 84 registros.
[
  {
    "segment": "Bens Indls / Máqs e Equips",
    "cod": "WEGE3",
    "asset": "WEG",
    "type": "ON  EJ  NM",
    "part": "2,905",
    "partAcum": "2,905",
    "theoricalQty": "1.482.105.837"
  },
  {
    "segment": "Bens Indls / Mat Transporte",
    "cod": "EMBR3",
    "asset": "EMBRAER",
    "type": "ON      NM",
    "part": "2,513",
    "partAcum": "2,751",
    "theoricalQty": "734.631.701"
  }
]


# Requisito 2: os dados brutos devem ser ingeridos no s3 em formato parquet com partição diária.

### GERAR PARQUET

In [3]:
import pandas as pd
from datetime import datetime
import os

# Data do pregão
data_pregao = datetime.today().strftime("%Y-%m-%d")

def process_registro(registro):
    # Corrigir campo 'segment'
    raw_segment = registro.get('segment', '')
    raw_segment = raw_segment.strip()
    setores = [s.strip() for s in raw_segment.split('/') if s.strip()] if raw_segment else []

    # Corrigir campo 'type'
    raw_type = registro.get('type', '')
    raw_type = raw_type.strip()
    tipos = [t.strip() for t in raw_type.split() if t.strip()] if raw_type else []

    # Adicionar listas e data
    registro['segment_list'] = setores
    registro['type_list'] = tipos
    registro['data_pregao'] = data_pregao
    return registro

# Processar
registros_processados = [process_registro(r) for r in registros]

# Criar DataFrame
df = pd.DataFrame(registros_processados)

# Nome do arquivo com data
output_dir = ".parquet"
filename = f"{output_dir}/dados_ibov_{data_pregao}.parquet"

# Criar a pasta se não existir
os.makedirs(output_dir, exist_ok=True)

# Salvar
df.to_parquet(filename, engine="pyarrow", index=False)
print(f"Arquivo salvo como: {filename}")


Arquivo salvo como: .parquet/dados_ibov_2025-06-21.parquet


### LER PARQUET

In [4]:
import pandas as pd
from pathlib import Path
import re

# Diretório onde os arquivos estão
caminho = Path('.')  # ou Path('caminho/da/pasta') se estiver em outro lugar

# Procurar arquivos parquet com nome no formato esperado
arquivos = list(caminho.glob(".parquet/dados_ibov_*.parquet"))

# Extrair data do nome e ordenar
def extrair_data(arquivo):
    match = re.search(r'dados_ibov_(\d{4}-\d{2}-\d{2})\.parquet', arquivo.name)
    if match:
        return match.group(1)
    return '0000-00-00'  # fallback para ordenar corretamente

# Ordenar do mais recente para o mais antigo
arquivos_ordenados = sorted(arquivos, key=extrair_data, reverse=True)

# Verifica se há arquivos
if arquivos_ordenados:
    arquivo_mais_recente = arquivos_ordenados[0]
    df = pd.read_parquet(arquivo_mais_recente)
    print(f"Abrindo: {arquivo_mais_recente.name}")
    print(df.head())
else:
    print("Nenhum arquivo 'dados_ibov_*.parquet' encontrado.")


Abrindo: dados_ibov_2025-06-21.parquet
                       segment    cod      asset        type   part partAcum  \
0   Bens Indls / Máqs e Equips  WEGE3        WEG  ON  EJ  NM  2,905    2,905   
1  Bens Indls / Mat Transporte  EMBR3    EMBRAER  ON      NM  2,513    2,751   
2  Bens Indls / Mat Transporte  POMO4  MARCOPOLO  PN      N2  0,238    2,751   
3        Bens Indls/Transporte  MOTV3  MOTIVA SA  ON      NM  0,632    1,927   
4        Bens Indls/Transporte  RAIL3  RUMO S.A.  ON  ED  NM  1,031    1,927   

    theoricalQty                  segment_list     type_list data_pregao  
0  1.482.105.837   [Bens Indls, Máqs e Equips]  [ON, EJ, NM]  2025-06-21  
1    734.631.701  [Bens Indls, Mat Transporte]      [ON, NM]  2025-06-21  
2    666.378.439  [Bens Indls, Mat Transporte]      [PN, N2]  2025-06-21  
3    991.920.937      [Bens Indls, Transporte]      [ON, NM]  2025-06-21  
4  1.216.914.397      [Bens Indls, Transporte]  [ON, ED, NM]  2025-06-21  
